In [21]:
import yfinance as yf
import pandas as pd

In [22]:
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    return stock

In [104]:
data = yf.download("^DJI 000001.SS ^GSPC", start = "2020-01-01")
data.dropna()
list(data.columns)

[*********************100%***********************]  3 of 3 completed


[('Adj Close', '000001.SS'),
 ('Adj Close', '^DJI'),
 ('Adj Close', '^GSPC'),
 ('Close', '000001.SS'),
 ('Close', '^DJI'),
 ('Close', '^GSPC'),
 ('High', '000001.SS'),
 ('High', '^DJI'),
 ('High', '^GSPC'),
 ('Low', '000001.SS'),
 ('Low', '^DJI'),
 ('Low', '^GSPC'),
 ('Open', '000001.SS'),
 ('Open', '^DJI'),
 ('Open', '^GSPC'),
 ('Volume', '000001.SS'),
 ('Volume', '^DJI'),
 ('Volume', '^GSPC')]

In [105]:
data["China SSE"] = (data["High", "000001.SS"] + data["Low", "000001.SS"]) / 2
data["US DJI"] = (data["High", "^DJI"] + data["Low", "^DJI"]) / 2
data["US S&P 500"] = (data["High", "^GSPC"] + data["Low", "^GSPC"]) / 2
data = data.drop(columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'])
data = data.dropna().reset_index()
data

,Date,China SSE,US DJI,US S&P 500
,,,,
0,2020-01-02,3082.218018,28750.285156,3246.834961
1,2020-01-03,3084.168579,28608.334961,3234.244995
2,2020-01-06,3086.255981,28563.325195,3230.739990
3,2020-01-07,3094.890015,28625.389648,3238.669922
4,2020-01-08,3076.685059,28694.344727,3251.869995
5,2020-01-09,3088.730103,28916.160156,3269.625000
6,2020-01-10,3093.310547,28899.084961,3271.925049
7,2020-01-13,3095.477051,28864.669922,3278.279907
8,2020-01-14,3116.386963,28963.214844,3285.719971


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.06,0.07,0.06,0.06,1031788800,0.0,0.0
1986-03-14,0.06,0.07,0.06,0.06,308160000,0.0,0.0
1986-03-17,0.06,0.07,0.06,0.07,133171200,0.0,0.0
1986-03-18,0.07,0.07,0.06,0.06,67766400,0.0,0.0
1986-03-19,0.06,0.06,0.06,0.06,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2020-03-13,147.50,161.91,140.73,158.83,92727400,0.0,0.0
2020-03-16,140.00,149.35,135.00,135.42,87905900,0.0,0.0
2020-03-17,140.00,147.50,135.00,146.57,81059800,0.0,0.0


In [ ]:
# Import data from Yahoo finance for selected indexes ^DJI, ^GSPC, .000001SS
data = yf.download("^DJI 000001.SS ^GSPC", start = "2020-01-01")
data.dropna()
data.head()

# Calculate Average Change between opening and closing price per day

data["China SSE"] = (data["High", "000001.SS"] + data["Low", "000001.SS"]) / 2
data["US DJI"] = (data["High", "^DJI"] + data["Low", "^DJI"]) / 2
data["US S&P 500"] = (data["High", "^GSPC"] + data["Low", "^GSPC"]) / 2
data = data.drop(columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'])
data = data.dropna().reset_index()
data


ITALY_DATASET = r"C:\Users\William\Documents\Github\Coronadex\Data\italytitansindex.csv"
italy_data = pd.read_csv(ITALY_DATASET)
italy_data.drop(columns= ["Unnamed: 2", "Unnamed: 3","Unnamed: 4","Unnamed: 5","Unnamed: 6","Unnamed: 7","Unnamed: 8"]).dropna()